### This script will import ABS population projections for Queensland and then export them into a seperate CSV for area and one combined CSV.
#### Install packages

In [1]:
install.packages("httr")
install.packages("readxl")
install.packages("here")
install.packages("dplyr")
install.packages("openssl")
install.packages("tidyverse")

Installing package into 'C:/Users/joshu/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'httr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\joshu\AppData\Local\Temp\RtmpeEptjq\downloaded_packages


Installing package into 'C:/Users/joshu/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'readxl' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'readxl'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\joshu\AppData\Local\R\win-library\4.4\00LOCK\readxl\libs\x64\readxl.dll to C:\Users\joshu\AppData\Local\R\win-library\4.4\readxl\libs\x64\readxl.dll: Permission denied"
Warning message:
"restored 'readxl'"


...

package 'openssl' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'openssl'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\joshu\AppData\Local\R\win-library\4.4\00LOCK\openssl\libs\x64\openssl.dll to C:\Users\joshu\AppData\Local\R\win-library\4.4\openssl\libs\x64\openssl.dll: Permission denied"
Warning message:
"restored 'openssl'"



The downloaded binary packages are in
	C:\Users\joshu\AppData\Local\Temp\RtmpeEptjq\downloaded_packages


Installing package into 'C:/Users/joshu/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'tidyverse' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\joshu\AppData\Local\Temp\RtmpeEptjq\downloaded_packages


In [2]:
library(httr)
library(readxl)
library(here)
library(dplyr)
library(openssl)
library(tidyverse)

here() starts at C:/Users/joshu


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Linking to: OpenSSL 3.3.0 9 Apr 2024

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.5.1     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


### Functions to download and extract file
### unzip_file
#### params
* zip_path - Path of the zipfile that will be processed
* dest_path - Path where processed file will go
### download_file
#### params
* url - String variable URL of file you wish to download
* dest_path - Destination of downloaded file
* extract_path - If zipped where the file will be extracted to
* file_name - What the saved file will be called

In [3]:
unzip_file <- function(zip_path, dest_path){
  unzip(zip_path, exdir = dest_path)
}

download_file <- function(url, dest_path, extract_path, file_name){
  # Ensure the destination directory exists
  if (!dir.exists(dest_path)) {
    dir.create(dest_path, recursive = TRUE)
  }
  
  full_dest_path <- file.path(dest_path, file_name)
  # Adjust extract_path to not append file_name directly
  GET(url, write_disk(full_dest_path, overwrite = TRUE))
  
  if (grepl(".zip$", full_dest_path)){
    unzip_file(full_dest_path, extract_path)
    file.remove(full_dest_path)
  }
}

In [4]:
url <- "https://api.data.abs.gov.au/data/ABS,POP_PROJ_REGION,1.0.0/32+31+3.2+1+3.TT.....A?startPeriod=2022&format=csv"
download_file(url, here("data"), here("data"), "ABS_PROJ_REGION_QLD.csv")
#import as dataframe

data <- read_csv(here("data", "ABS_PROJ_REGION_QLD.csv"))
data_df <- as.data.frame(data)

Rows: 32400 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): DATAFLOW, AGE, FREQUENCY, UNIT_MEASURE
dbl (8): REGION, SEX_ABS, FERTILITY, MORTALITY, NOM, NIM, TIME_PERIOD, OBS_V...
lgl (2): OBS_STATUS, OBS_COMMENT

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


#### Select relevant columns and add metadata before creating regional dataframes

In [8]:
colnames(data_df)
data_df <- data_df %>% select(REGION, FERTILITY, SEX_ABS, MORTALITY, NOM, NIM, TIME_PERIOD, OBS_VALUE)
#split the data by region, region 32 is brisbane, region 3 is rest of qld and remove the REGION column
#create a primary key column using ssh for the data using the OBS_VALUE column
data_df$GROWTH_KEY <- md5(as.character(data_df$OBS_VALUE))
data_df$DATE_RETRIEVED <- today()
#filter y region
qld_df <- data_df %>% filter(REGION == 3) %>% select(-REGION)
brisbane_df <- data_df %>% filter(REGION == 32) %>% select(-REGION)
rest_of_qld_df <- data_df %>% filter(REGION == 31) %>% select(-REGION)

[1] "REGION"         "FERTILITY"      "SEX_ABS"        "MORTALITY"     
 [5] "NOM"            "NIM"            "TIME_PERIOD"    "OBS_VALUE"     
 [9] "GROWTH_KEY"     "DATE_RETRIEVED"

Warning message in with_tz.default(Sys.time(), tzone):
"Unrecognized time zone ''"


#### Write to CSVs

In [ ]:
write.csv(data_df, "abs_pop_proj_data_all.csv")
write.csv(qld_df, "abs_pop_proj_data_qld.csv")
write.csv(brisbane_df, "abs_pop_proj_data_brisbane.csv")
write.csv(rest_of_qld_df, "abs_pop_proj_data_rest_of_qld_df.csv")

### Classifications
#### Region
- 3 - QLD
- 31 - Rest of QLD
- 32 - Brisbane
#### Fertility

<ol>
<li>High fertility</li>
<li>Medium fertility</li>
<li>Low fertility</li>
</ol>

#### Sex
<ol>
<li>Males</li>
<li>Females</li>
<li>Persons</li>
</ol>

#### Mortality
<ol>
<li>High life expectancy</li>
<li>Medium life expectancy</li>
</ol>

#### NOM Net Overseas Migration
<ol>
<li>High NOM</li>
<li>Medium NOM</li>
<li>Low NOM</li>
<li>Zero NOM</li>
</ol>

#### NIM Net Interstate Migration
<ol>
<li>Large interstate flows</li>
<li>Medium interstate flows</li>
<li>Small interstate flows</li>
</ol>

#### Time Period
TIME_PERIOD - Prediction year
#### OBS_VALUE
OBS_VALUE - Predicted population